In [2]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

'''
Authenticate
Authenticates your credentials and creates a client.
pip install azure-cognitiveservices-vision-computervision
'''
subscription_key = ""
endpoint = ""

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

'''
OCR: Read File using the Read API, extract text - remote
This example will extract text in an image, then print results, line by line.
This API call can also extract handwriting style text (not shown).
'''
print("===== Read File - remote =====")
# Get an image with text
read_image_url = "https://stat.ameba.jp/user_images/20140221/16/koumuinsiken-yobikou/86/3b/p/o0471068512853433393.png"

# Call API with URL and raw response (allows you to get the operation location)
read_response = computervision_client.read(read_image_url,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()

===== Read File - remote =====
【地方上級過去問 1】次の文の主旨として、最も妥当なのはどれか。
[37.0, 38.0, 351.0, 38.0, 351.0, 51.0, 37.0, 51.0]
日本人は常に外国の文化を尊敬と親愛をもって、自国に取り入れ、時には自国の民族性を
[34.0, 56.0, 443.0, 56.0, 443.0, 69.0, 34.0, 69.0]
見失うように見える時期もありますが、それを消化吸収して、日本独自のものを生み出し
[39.0, 73.0, 443.0, 73.0, 443.0, 86.0, 39.0, 86.0]
て来ていることは皆様も認めておられると思います。
[39.0, 92.0, 278.0, 92.0, 278.0, 104.0, 39.0, 104.0]
丁度、今、日本の昔の書の展覧会がケルンで催されていますが、中国の漢字を基にしなが
[32.0, 109.0, 444.0, 108.0, 444.0, 121.0, 32.0, 121.0]
ら、千年も前に、すでに仮名という日本独特の文字を造り出しているのを御覧になったで
[40.0, 125.0, 443.0, 125.0, 443.0, 138.0, 40.0, 138.0]
しょう。仮名の美、あるいは、漢字と仮名の調和の美は、日本民族が生み出したものであ
[41.0, 144.0, 443.0, 144.0, 443.0, 156.0, 41.0, 156.0]
ります。
[39.0, 161.0, 73.0, 161.0, 74.0, 173.0, 40.0, 173.0]
美術の分野でも千年も前から漢絵––中国の絵画または中国風な絵––と、大和絵––日
[31.0, 179.0, 443.0, 178.0, 444.0, 193.0, 31.0, 194.0]
本の民族的な好みによって生まれた絵画形式––との二つが行われて来ました。また、明
[37.0, 196.0, 444.0, 196.0, 445.0, 208.0, 37.0, 208.0]
治期以後、西洋からの油絵と、日本の伝統的な画風の日本画との二つが現在まで続いてい
[38.0, 213.0, 444.0, 213.0, 444.0, 225.0, 38

In [5]:
'''
OCR: Read File using the Read API, extract text - local
This example extracts text from a local image, then prints results.
This API call can also recognize remote image text (shown in next example, Read File - remote).
'''
print("===== Read File - local =====")

images_folder = "/workspace/products"
# Get image path
read_image_path = os.path.join (images_folder, "sentence.png")
# Open the image
read_image = open(read_image_path, "rb")

# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(10)

# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()
'''
END - Read File - local
'''

print("End of Computer Vision quickstart.")

===== Read File - local =====
Waiting for result...
【地方上級過去問 1】次の文の主旨として、最も妥当なのはどれか。
[37.0, 38.0, 351.0, 38.0, 351.0, 51.0, 37.0, 51.0]
日本人は常に外国の文化を尊敬と親愛をもって、自国に取り入れ、時には自国の民族性を
[34.0, 56.0, 443.0, 56.0, 443.0, 69.0, 34.0, 69.0]
見失うように見える時期もありますが、それを消化吸収して、日本独自のものを生み出し
[39.0, 73.0, 443.0, 73.0, 443.0, 86.0, 39.0, 86.0]
て来ていることは皆様も認めておられると思います。
[39.0, 92.0, 278.0, 92.0, 278.0, 104.0, 39.0, 104.0]
丁度、今、日本の昔の書の展覧会がケルンで催されていますが、中国の漢字を基にしなが
[32.0, 109.0, 444.0, 108.0, 444.0, 121.0, 32.0, 121.0]
ら、千年も前に、すでに仮名という日本独特の文字を造り出しているのを御覧になったで
[40.0, 125.0, 443.0, 125.0, 443.0, 138.0, 40.0, 138.0]
しょう。仮名の美、あるいは、漢字と仮名の調和の美は、日本民族が生み出したものであ
[41.0, 144.0, 443.0, 144.0, 443.0, 156.0, 41.0, 156.0]
ります。
[39.0, 161.0, 73.0, 161.0, 74.0, 173.0, 40.0, 173.0]
美術の分野でも千年も前から漢絵––中国の絵画または中国風な絵––と、大和絵––日
[31.0, 179.0, 443.0, 178.0, 444.0, 193.0, 31.0, 194.0]
本の民族的な好みによって生まれた絵画形式––との二つが行われて来ました。また、明
[37.0, 196.0, 444.0, 196.0, 445.0, 208.0, 37.0, 208.0]
治期以後、西洋からの油絵と、日本の伝統的な画風の日本画との二つが現在まで続いてい
[38.0, 213.0, 444.0, 21